In [1]:
!pip install basedosdados

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os
import zipfile
from zipfile import ZipFile
import basedosdados as bd

### Setup inicial
_necessário rodar apenas uma vez_

* Download dos arquivos  
* Criação das pastas de partição (ano) em output
* Como os municípios são listados por nome, substituímos pelo id_municipio (ID Município - IBGE 7 Dígitos). Tal check cria a lista de todos os municípios que não dão match com a base de compatibilização extraída do diretório de município. Baseado nessa lista, faz-se o replace para um match 1:1

In [ ]:
# download dos dados originais
for ano in [*range(2003, 2023)]:
  !wget --no-check-certificate -P /content/drive/MyDrive/basedosdados/br_inpe_queimadas/input https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/focos_br_ref_{ano}.zip

In [ ]:
# cria pastas particionadas por ano
for ano in [*range(2003, 2023)]:
  directory = '/content/drive/MyDrive/basedosdados/br_inpe_queimadas/output/ano={}'.format(ano)
  if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
# compatibilização da base entre nomes, siglas e ids
query = '''
SELECT
  sigla_uf,
  UPPER(nome_uf) AS estado,
  id_municipio,
  UPPER(nome) AS municipio
FROM basedosdados.br_bd_diretorios_brasil.municipio
'''

comp = bd.read_sql(query, billing_project_id='input-dados')

In [ ]:
# check de match para municipios
unmatch_list = []

for ano in [*range(2003, 2023)]:
    file = '/content/drive/MyDrive/basedosdados/br_inpe_queimadas/input/focos_br_ref_{}.zip'.format(ano)
    # descompacta o arquivo csv
    with ZipFile(file) as z:
      with z.open('focos_br_ref_{}.csv'.format(ano)) as f:
        df = pd.read_csv(f)
        # merge com diretorio
        df = pd.merge(df, comp, how='left', on=['estado', 'municipio'], indicator=True)
        # cria lista de municipios com typos
        unmatch = df.query('_merge == "left_only"')['municipio'].drop_duplicates().to_list()
        # join entre anos
        unmatch_list = unmatch_list + unmatch

# função que remove duplicados na lista
def remove_repetidos(lista):
    l = []
    for i in lista:
        if i not in l:
          l.append(i)
    l.sort()
    return l

unmatch_list_unique = remove_repetidos(unmatch_list) #lista conta com 26 municipios e typos para dados até 2022

### Tratamento

In [ ]:
for ano in [*range(2003, 2023)]:
  #if ano == 2003:
    file = '/content/drive/MyDrive/basedosdados/br_inpe_queimadas/input/focos_br_ref_{}.zip'.format(ano)
    # descompacta o arquivo csv
    with ZipFile(file) as z:
      with z.open('focos_br_ref_{}.csv'.format(ano)) as f:
        df = pd.read_csv(f)
        df['municipio'].replace({'ARAÇÁS':'ARAÇAS',
                                'ATÍLIO VIVACQUA':'ATILIO VIVACQUA',
                                'AUGUSTO SEVERO':'CAMPO GRANDE',
                                'BIRITIBA MIRIM':'BIRITIBA-MIRIM',
                                'FLORÍNEA':'FLORÍNIA',
                                'IGUARACY':'IGUARACI',
                                'ITAOCA':'ITAÓCA',
                                'ITAPAJÉ':'ITAPAGÉ',
                                'IUIU':'IUIÚ',
                                'JANUÁRIO CICCO':'BOA SAÚDE',
                                'LAURO MÜLLER':'LAURO MULLER',
                                'MUQUÉM DO SÃO FRANCISCO':'MUQUÉM DE SÃO FRANCISCO',
                                "OLHO D'ÁGUA DO BORGES":"OLHO-D'ÁGUA DO BORGES",
                                'PASSA VINTE':'PASSA-VINTE',
                                "PINGO D'ÁGUA":"PINGO-D'ÁGUA",
                                'POXORÉU':'POXORÉO',
                                'RESTINGA SÊCA':'RESTINGA SECA',
                                'SANTA IZABEL DO PARÁ':'SANTA ISABEL DO PARÁ',
                                'SÃO CRISTÓVÃO DO SUL':'SÃO CRISTOVÃO DO SUL',
                                'SÃO LUIZ DO NORTE':'SÃO LUÍZ DO NORTE',
                                'SÃO LUIZ DO PARAITINGA':'SÃO LUÍS DO PARAITINGA',
                                'SÃO VICENTE FÉRRER':'SÃO VICENTE FERRER',
                                'VESPASIANO CORRÊA':'VESPASIANO CORREA',
                                'WESTFÁLIA':'WESTFALIA'}, inplace=True)
        df['municipio'] = np.where((df['estado'] == 'BAHIA') & (df['municipio'] == 'SANTA TEREZINHA'), 'SANTA TERESINHA', df['municipio'])
        df['municipio'] = np.where((df['estado'] == 'PARAÍBA') & (df['municipio'] == 'QUIXABA'), 'QUIXABÁ', df['municipio'])
        df = pd.merge(df, comp, how='left', on=['estado', 'municipio'], indicator=True)
        df.rename(columns={'lat':'latitude', 'lon':'longitude', 'data_pas':'data_hora', 'foco_id':'id_foco'}, inplace=True)
        df = df[['sigla_uf', 'id_municipio', 'bioma', 'id_bdq', 'id_foco', 'data_hora', 'latitude', 'longitude']]
        df.to_csv('/content/drive/MyDrive/basedosdados/br_inpe_queimadas/output/ano={}/microdados.csv'.format(ano), index=False, na_rep='')